In [22]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.calibration import LabelEncoder
import pickle 

In [23]:
data  = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [24]:
data = data.drop(['RowNumber',"CustomerId","Surname"],axis=1)


In [25]:
le = LabelEncoder()
data['Gender'] = le.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [26]:
ohe = OneHotEncoder()
geo_enc = ohe.fit_transform(data[['Geography']])
geo_enc

<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [27]:
geo_enc.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [28]:
ohe.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [29]:
geo_enc_df = pd.DataFrame(geo_enc.toarray(),columns=ohe.get_feature_names_out(['Geography']))

In [30]:
data = pd.concat([data.drop('Geography',axis=1),geo_enc_df],axis=1)

In [31]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [32]:
with open('label_encoder_gender.pkl','wb')  as f:
    pickle.dump(le,f)
with open('ohe_geo.pkl','wb')  as f:
    pickle.dump(ohe,f)



In [33]:
X = data.drop('Exited',axis=1)
Y = data['Exited']

In [43]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=43)
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)

In [44]:
X_train

array([[ 1.5178518 ,  0.90134069, -0.85641715, ...,  1.00125078,
        -0.57715782, -0.57850497],
       [ 1.0209782 ,  0.90134069, -0.28681727, ..., -0.99875078,
        -0.57715782,  1.72859362],
       [ 2.02507693, -1.1094584 , -0.09695065, ..., -0.99875078,
        -0.57715782,  1.72859362],
       ...,
       [ 0.48269847, -1.1094584 , -1.14121709, ..., -0.99875078,
         1.73262835, -0.57850497],
       [ 0.66902607,  0.90134069,  0.56758255, ..., -0.99875078,
        -0.57715782,  1.72859362],
       [-0.27296346,  0.90134069, -0.57161721, ..., -0.99875078,
         1.73262835, -0.57850497]])

In [45]:
with open('scalar.pkl','wb') as f:
    pickle.dump(scalar,f)

In [46]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [47]:
# build model

model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
])

In [48]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [49]:
import tensorflow as tf
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
loss = tf.keras.losses.BinaryCrossentropy()

In [50]:
model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

In [51]:
log_dir = 'logs/fit/'
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [53]:
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)
history = model.fit(
    X_train,Y_train,validation_data=(X_test,Y_test),epochs=100,
    callbacks = [tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5644 - accuracy: 0.7770 - val_loss: 0.5054 - val_accuracy: 0.7950
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4828 - accuracy: 0.7981 - val_loss: 0.4671 - val_accuracy: 0.7995
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4537 - accuracy: 0.8034 - val_loss: 0.4490 - val_accuracy: 0.8065
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4368 - accuracy: 0.8104 - val_loss: 0.4384 - val_accuracy: 0.8120
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4258 - accuracy: 0.8150 - val_loss: 0.4319 - val_accuracy: 0.8155
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4179 - accuracy: 0.8188 - val_loss: 0.4265 - val_accuracy: 0.8230
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4114 - accuracy: 0.8226 - val_loss: 0.4219 - val_accuracy: 0.8265

In [54]:
model.save('model.h5')

c:\Users\Sudeep\Documents\Complete GenAI\Project-1 ANN CLASSIFICATION\.venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [55]:
%load_ext tensorboard

In [56]:
%tensorboard --logdir logs/fit